In [ ]:
# default_exp uid3

# UId3
> UId3

In [ ]:
from graphviz import Source

from pyuid3.attribute import Attribute
from pyuid3.data import Data
from pyuid3.entropy_evaluator import EntropyEvaluator
from pyuid3.uncertain_entropy_evaluator import UncertainEntropyEvaluator
from pyuid3.tree import Tree
from pyuid3.tree_node import TreeNode
from pyuid3.tree_edge import TreeEdge
from pyuid3.value import Value

In [ ]:
# export
class UId3:
    NODE_SIZE_LIMIT = 1
    TREE_DEPTH_LIMIT = 2
    GROW_CONFIDENCE_THRESHOLD = 0
    
    @staticmethod
    def grow_tree(data, entropyEvaluator, depth: int):
        if len(data.get_instances()) < UId3.NODE_SIZE_LIMIT:
            return None
        if depth > UId3.TREE_DEPTH_LIMIT:
            return None
        entropy = UncertainEntropyEvaluator().calculate_entropy(data)
    
        data.update_attribute_domains()

        # of the set is heterogeneous or no attributes to split, just class -- return
        # leaf
        if entropy == 0 or len(data.get_attributes()) == 1:
            # create the only node and summary for it
            class_att = data.get_class_attribute()
            root = TreeNode(class_att.get_name(), data.calculate_statistics(class_att))
            root.set_type(class_att.get_type())
            result = Tree(root)
            return result

        info_gain = 0
        best_split = None
        for a in data.get_attributes():
            if data.get_class_attribute() == a:
                continue 
            values = a.get_domain()
            temp_gain = entropy
            temp_numeric_gain = 0
            stats = data.calculate_statistics(a)
            for v in values:
                subdata = None
                subdataLessThan = None
                subdataGreaterEqual = None
                if a.get_type() == Attribute.TYPE_NOMINAL:
                    subdata = data.filter_nominal_attribute_value(a, v)
                elif a.get_type() == Attribute.TYPE_NUMERICAL:
                    subdata_less_than = data.filter_numeric_attribute_value(a, v, True)
                    subdata_greater_equal = data.filter_numeric_attribute_value(a, v, False)
                
                if a.get_type() == Attribute.TYPE_NOMINAL:
                    temp_gain -= stats.get_stat_for_value(v) * UncertainEntropyEvaluator().calculate_entropy(subdata)
                elif a.get_type() == Attribute.TYPE_NUMERICAL:
                    single_temp_gain = entropy - stats.get_stat_for_value(v) * (UncertainEntropyEvaluator().calculate_entropy(subdata_less_than) + UncertainEntropyEvaluator().calculate_entropy(subdata_greater_equal))
                    if single_temp_gain >= temp_numeric_gain:
                        temp_numeric_gain = single_temp_gain
                        temp_gain = single_temp_gain
                        a.set_value_to_split_on(v)

            if temp_gain >= info_gain:
                info_gain = temp_gain
                best_split = a
                a.set_importance_gain(info_gain)

        # if nothing better can happen
        if best_split == None:
            # create the only node and summary for it
            class_att = data.get_class_attribute()
            root = TreeNode(class_att.get_name(), data.calculate_statistics(class_att))
            root.set_type(class_att.get_type())
            result = Tree(root)
            return result

        # Create root node, and recursively go deeper into the tree.
        class_att = data.get_class_attribute()
        class_stats = data.calculate_statistics(class_att)
        root = TreeNode(best_split.get_name(), class_stats)
        root.set_type(class_att.get_type())

        # attach newly created trees
        for val in best_split.get_splittable_domain():
            if best_split.get_type() == Attribute.TYPE_NOMINAL:
                new_data = data.filter_nominal_attribute_value(best_split, val)
                subtree = UId3.grow_tree(new_data, EntropyEvaluator, depth + 1)
                best_split_stats = data.calculate_statistics(best_split)
                if subtree and best_split_stats.get_most_probable().get_confidence() > UId3.GROW_CONFIDENCE_THRESHOLD:
                    root.add_edge(TreeEdge(Value(val, best_split_stats.get_avg_confidence()), subtree.get_root()))
                    root.set_infogain(best_split.get_importance_gain())
                    
            elif best_split.get_type() == Attribute.TYPE_NUMERICAL:
                new_data_less_then = data.filter_numeric_attribute_value(best_split, val, True)
                new_data_greater_equal = data.filter_numeric_attribute_value(best_split, val, False)
                subtree_less_than = UId3.grow_tree(new_data_less_then, EntropyEvaluator, depth + 1)
                subtree_greater_equal = UId3.grow_tree(new_data_greater_equal, EntropyEvaluator, depth + 1)
                best_split_stats = data.calculate_statistics(best_split)
                
                if subtree_less_than and best_split_stats.get_most_probable().get_confidence() > UId3.GROW_CONFIDENCE_THRESHOLD:
                    root.add_edge(TreeEdge(Value("<" + val, best_split_stats.get_avg_confidence()), subtree_less_than.get_root()))
                if subtree_greater_equal and best_split_stats.get_most_porbable().get_confidence() > UId3.GROW_CONFIDENCE_THRESHOLD:
                    root.add_edge(TreeEdge(Value(">=" + val, best_split_stats.get_avg_confidence()), subtree_greater_equal.get_root()))
                root.set_type(Attribute.TYPE_NUMERICAL)
                root.set_infogain(best_split.get_importance_gain())
                
        if len(root.get_edges()) == 0:
            root.set_att(data.get_class_attribute().get_name())
            root.set_type(data.get_class_attribute().get_type())
            
        return Tree(root)

In [ ]:
# export
def main(type_t="dot"):
    data = Data.parse_uarff("/Users/ola/Develop/udt/resources/weather.nominal.uncertain.arff")
    t = UId3.grow_tree(data, UncertainEntropyEvaluator(), 0)

    if type_t == "dot":
        result = t.to_dot()
        print(result)
        s = Source(result, filename="test.gv", format="png")
        s.view()
    elif type_t == "str": 
        print(str(t))
    elif type_t == "hmr":
        print(t.to_HMR())
    elif type_t == "infogain":
        print(t.get_importances())


In [ ]:
# export
if __name__ == "__main__":
    main()

weather.symbolic
sunny[0.5]
rainy[0.3]
overcast[0.2]
hot
high
FALSE
no
sunny
hot
high
TRUE
no
overcast[0.2]
sunny[0.7]
rainy[0.1]
hot
high
FALSE
yes
rainy
mild
high
FALSE
yes
rainy
cool
normal
FALSE[0.3]
yes
rainy
cool
normal
TRUE
no
overcast
cool
normal
TRUE
yes
sunny
mild
high
FALSE[0.6]
TRUE[0.4]
no
sunny
cool
normal[0.4]
FALSE
yes
rainy
mild
normal
FALSE
yes
sunny
mild
normal
TRUE
yes
overcast
mild
high
TRUE[0.4]
yes
overcast
hot
normal
FALSE
yes
rainy
mild
high
TRUE
no
digraph mediationTree{
8763654589465[label=" outlook
 ",shape=box, color=black]8763654589465->8763654595524[label="rainy
 conf=0.94 "]
8763654595524[label=" windy
 ",shape=box, color=black]8763654595524->8763654595551[label="FALSE
 conf=0.94 "]
8763654595551[label=" play
yes[1.0]
no[0.0]
 ",shape=box, color=red]8763654595524->8763654595512[label="TRUE
 conf=0.94 "]
8763654595512[label=" play
no[0.67]
yes[0.33]
 ",shape=box, color=red]8763654589465->8763654595398[label="overcast
 conf=0.94 "]
8763654595398[label=" pl